In [1]:
import tensorflow as tf
from tensorflow.contrib.data import Dataset
import glob
import sys
import imageio
import scipy
import numpy as np

In [2]:
# helper functions
def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [3]:
#functions to load data

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((num_classes, y.shape[0]))
    y_one_hot[y, range(y.shape[0])] = 1
    return y_one_hot

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [4]:
data_path = "cifar10-hw2/"
X_train, y_train = get_train_data(data_path)

{'airplane': 0, 'automobile': 1, 'dog': 5, 'deer': 4, 'cat': 3, 'ship': 8, 'horse': 7, 'truck': 9, 'bird': 2, 'frog': 6}


/home/leo/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  


Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000


In [5]:
# step 2: create a dataset returning slices of `filenames`  https://stackoverflow.com/questions/37340129/tensorflow-training-on-my-own-image
tf.reset_default_graph()
"""
def train_input_fn(features, labels, batch_size):
    
    dataset = tf.data.Dataset.from_tensor_slices((features,labels))
    # step 3: parse every image in the dataset using `map`
    dataset = dataset.map(_parse_function)
    print(dataset)
    #dataset = dataset.batch(batch_size)

    iterator = dataset.make_one_shot_iterator()
    features, labels = iterator.get_next()
    print((features,labels))

    return features, labels
   
def _parse_function(filename,label):
    print(filename)
    print("HELLO")
    image_string = tf.read_file(filename)
    print(image_string)
    image_decoded = tf.image.decode_png(image_string, channels=3)
    image_decoded = tf.reshape(image_decoded, [3072])

    image = tf.cast(image_decoded, tf.float32)
    print(image)
    label = tf.reshape(label, [1])
    return image,label"""

def my_model_fn(
   features, # This is batch_features from input_fn
   labels,   # This is batch_labels from input_fn
   mode,     # An instance of tf.estimator.ModeKeys
   params):  # Additional configuration
    
    input_layer = tf.reshape(features["x"],[-1,32,32,3])
    input_layer = tf.cast(input_layer,dtype=tf.float32)
    
    #conv layer 1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[2,2],
      padding="same",
      activation=tf.nn.relu)
    
    #pooling layer 1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size = [2,2],strides = 2)
    
    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(inputs = pool1, filters = 64, kernel_size = [2,2],
                             padding = 'same', activation = tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs = conv2, pool_size=[2,2],strides = 2)
    print(pool2)
    #dense layer
    pool2_flat = tf.reshape(pool2, [-1, 8*8*64])
    print(pool2_flat)
    dense = tf.layers.dense(inputs=pool2_flat, units=512, activation=tf.nn.relu)
    print(dense)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    logits = tf.layers.dense(inputs = dense, units = 10)
    print(logits)
    
    if(mode == tf.estimator.ModeKeys.PREDICT):
        predictions = {
            'class_ids' : tf.argmax(inputs=logits,axis=1),
            'probabilities' : tf.nn.softmax(logits,name = "softmax_tensor")
            
        }
        return tf.estimator.EstimatorSpec(mode,predictions=predictions)


    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    if mode == tf.estimator.ModeKeys.EVAL:
        eval_metric_ops = {
            "accuracy": tf.metrics.accuracy(
                labels=labels, predictions=predictions["classes"])}
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
    
        optimizer = tf.train.RMSPropOptimizer(learning_rate=.1)

        train_op = optimizer.minimize(loss, global_step = tf.train.get_global_step())

        return tf.estimator.EstimatorSpec(mode,loss=loss, train_op=train_op)
                     

In [6]:
feature_columns = [tf.feature_column.numeric_column("x", shape=[3072], dtype=tf.float32)]
tf.reset_default_graph()
#set up classifier
classifier = tf.estimator.Estimator(
    model_fn = my_model_fn,
    #model_dir = "model_tmp/",
    params={
        'feature_columns' : feature_columns,
        #choose between 10 classes
        'n_classes':10,
        
    }
)

X_train = X_train.reshape(50000,3072)
y_train = y_train.reshape(50000,)
print(X_train.shape)
print(y_train.shape)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x":X_train},
    y = y_train,
    batch_size = 300,
    num_epochs = None,
    shuffle = True
    )

classifier.train(input_fn=train_input_fn, steps = 5000)








INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_worker_replicas': 1, '_save_summary_steps': 100, '_task_id': 0, '_is_chief': True, '_save_checkpoints_steps': None, '_num_ps_replicas': 0, '_service': None, '_tf_random_seed': None, '_task_type': 'worker', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe1f4352da0>, '_session_config': None, '_save_checkpoints_secs': 600, '_master': '', '_model_dir': '/tmp/tmpf99wyd2s', '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000}
(50000, 3072)
(50000,)
Tensor("max_pooling2d_2/MaxPool:0", shape=(300, 8, 8, 64), dtype=float32)
Tensor("Reshape_1:0", shape=(300, 4096), dtype=float32)
Tensor("dense/Relu:0", shape=(300, 512), dtype=float32)
Tensor("dense_2/BiasAdd:0", shape=(300, 10), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpf99wyd2s/model.ckpt.
INFO:tensorflow:loss = 2.321501, 

KeyboardInterrupt: 